**Connecting to a Device**
======

In this tutorial we will connect to some EPICS IOCs. You can veiw the database [here](https://gitlab.helmholtz-berlin.de/sissy/ioc/emilel/EMIL_PZI_00/-/blob/multiAxisPiezo/EMIL_PZI_00App/Db/singleAxis.db) This template is used to create an IOC.

There are three PV's 

```
EMILEL:TestIOC00:axis1:getPos
EMILEL:TestIOC00:axis1:done
EMILEL:TestIOC00:axis1:setPos
```

This represents a simple axis where we can define a set point, read a readback and monitor when the axis has moved. The IOC uses a calc record to make the ```getPos``` slowly reach the ```setPos```. 

Below we group these three PV's logically together into a single class. We Inherit from the [PVPositioner Class](https://nsls-ii.github.io/ophyd/positioners.html#pvpositioner)

In [ ]:
from ophyd import PVPositioner, EpicsSignal, EpicsSignalRO, Device, DerivedSignal,PseudoPositioner,PseudoSingle, Signal
from ophyd import Component as Cpt



## Explore creating a device from other devices

class Axis(PVPositioner):
  
    setpoint = Cpt(EpicsSignal,'setPos') 
    readback = Cpt(EpicsSignalRO,'getPos', kind='hinted')     # the 'hinted' flag is used by plotting tools to hint that we want to plot this thing
    done = Cpt(EpicsSignal, 'done')
    
   

We can then create an instance of that class

In [ ]:
prefix = 'EMILEL:TestIOC00:axis1:'
my_simple_robot = Axis(prefix, name='single_axis')  # The name is what's used in metadata or documents to refer to this thing

my_simple_robot.wait_for_connection()

And use the ```summary()``` method to tell us what's in it

In [ ]:
my_simple_robot.summary()

We can get the current value.

In [ ]:
x = my_simple_robot.readback.get()
print(x)

To illistrate that devices have a [status](https://nsls-ii.github.io/ophyd/status.html), we can create a callback, set the position and then observe the callback triggering some time later. 

In [ ]:
def callback(status):
 
    print(f"{status} is done")
    
my_simple_robot.set(x*0.5).add_callback(callback)


**The Device Class**
======

PVPositioner is based on the base class [Device](https://nsls-ii.github.io/ophyd/device-overview.html#device) which contains the methods like ```summary()```, ```connected()```, ```set()```, ```get()``` and ```trigger()```

The [IOC actually creates three axis](https://gitlab.helmholtz-berlin.de/sissy/ioc/emilel/EMIL_PZI_00/-/blob/multiAxisPiezo/iocBoot/iocEMIL_PZI_00/st.cmd#L20) 

```
EMILEL:TestIOC00:axis1:getPos
EMILEL:TestIOC00:axis1:done
EMILEL:TestIOC00:axis2:getPos
EMILEL:TestIOC00:axis2:done
EMILEL:TestIOC00:axis3:getPos
EMILEL:TestIOC00:axis3:done
EMILEL:TestIOC00:axis1:setPos
EMILEL:TestIOC00:axis2:setPos
EMILEL:TestIOC00:axis3:setPos
```

We can make a device, which contains other devices in this case the Axis class we just created.

In [ ]:
class ThreeAxis(Device):
    x = Cpt(Axis, 'axis1:')
    y = Cpt(Axis, 'axis2:')
    z = Cpt(Axis, 'axis3:')
    
prefix = 'EMILEL:TestIOC00:'
my_three_axis_robot = ThreeAxis(prefix, name='robot')

# Wait for a connection before we continue.
my_three_axis_robot.wait_for_connection()


Devices can be complex like the one above, or simple where we just want to read the status of somthing. The BESSY II ring is a good example. We never want to modify these parameters, but they are important to read for metadata collection

In [ ]:
class ring(Device):
    
    """
    Object to query machine BESSY II Beam status.

    """

    current  = Cpt(EpicsSignalRO,      'current'   ,kind="hinted" )
    lifetime = Cpt(EpicsSignalRO,      'lt10'      ,kind="hinted" )

bessy_2 = ring('MDIZ3T5G:', name='bessy_2')
bessy_2.wait_for_connection()

bessy_2.read()


**PV Name Construction**
======
Note how the PV names are built sequentially, from the prefix and then at each level as a new device is added, until finally we reach the signal. This works well when the PV name has a structure with common parts running through it. This is not the case for a lot of PV's at BESSY II. 

Consider the 4 motors of tha SISSY AU3 aperture unit. 

```
AUY02U112L:AbsM1
AUY02U112L:rdPosM1
AUY02U112L:AbsM2
AUY02U112L:rdPosM2
AUY02U112L:AbsM3
AUY02U112L:rdPosM3
AUY02U112L:AbsM4
AUY02U112L:rdPosM4
AUY02U112L:state

```

The common parts of the name are the prefix ```AUY02U112L:``` and a suffix denoting the Motor number. The ```state``` pv is used for the done signal. We can construct a PV name using the FormattedComponent Class. The snippet below is taken from the beamlineOphydDevices [Axes](https://gitlab.helmholtz-berlin.de/sissy/experiment-control/beamlineOphydDevices/-/blob/master/axes.py) and [Aperture](https://gitlab.helmholtz-berlin.de/sissy/experiment-control/beamlineOphydDevices/-/blob/master/AU.py) device definitions

In [ ]:
from ophyd import FormattedComponent as FCpt

# Used on AU1, AU3 and Pinhole        
class AxisTypeA(PVPositioner):

    setpoint = FCpt(EpicsSignal,    '{self.prefix}Abs{self._ch_name}'                 )                   
    readback = FCpt(EpicsSignalRO,  '{self.prefix}rdPos{self._ch_name}', kind='hinted')
    done     = FCpt(EpicsSignalRO,  '{self.prefix}State{self._ch_name}'               )
    
    done_value = 0   
    
    def __init__(self, prefix, ch_name=None, **kwargs):
        self._ch_name = ch_name
        super().__init__(prefix, **kwargs)


#Can be used for AU1 or AU3 units
class AU13(Device):

    read_attrs  = ['top.readback', 'bottom.readback', 'bottom.left', 'bottom.right']  
    top         = Cpt(AxisTypeA, '', ch_name='M1')
    bottom      = Cpt(AxisTypeA, '', ch_name='M2')
    left        = Cpt(AxisTypeA, '', ch_name='M3') # wall in old convention
    right       = Cpt(AxisTypeA, '', ch_name='M4') #ring in old convention

sissy_au3 = AU13('AUY02U112L:', name='sissy_au3')

sissy_au3.wait_for_connection()

#Show that we have connected 
if(sissy_au3.connected):
    print("Every PV name has been resolved and is connected to real hardware")
else:
    print("somthing is wrong!")

**Connecting to Detectors**
====

So far we have only connected to devices that we want to move, but we alse need to be able to create devices used in bluesky plans that read values. These might be as simple as an electrometer reading a current from a photodiode, or as complicated as a CCD. The latter is made easier by [Ophyd support for EPICS AreaDetector Devices.](https://nsls-ii.github.io/ophyd/area-detector.html#area-detectors)

Going back to our earlier simulated IOC with three axis, we now introduce an additional PV which is dependant on the position of those axis

```
EMILEL:TestIOC00:sensor1:getCount
EMILEL:TestIOC00:sensor2:getCount
EMILEL:TestIOC00:sensor3:getCount
```

Let's make a trivial device to expose these signals

In [ ]:
class Sensor(Device):
    
    counts = Cpt(EpicsSignalRO,  'getCount', kind='hinted')
    
prefix = 'EMILEL:TestIOC00:'
my_sensor_1 = Sensor(prefix+'sensor1:', name='sensor_1')

my_sensor_1.wait_for_connection()

In [ ]:
my_sensor_1.read()


**Putting it all together**
=====

Now we can use bluesky to scan the values of the x axis of our robot and observe what happens to the detector. In the example below we just use a temporary database and output to a livetable for ease.

In [ ]:
from bluesky import RunEngine
RE = RunEngine({})

from bluesky.callbacks.best_effort import BestEffortCallback
bec = BestEffortCallback()

# Send all metadata/data captured to the BestEffortCallback.
RE.subscribe(bec)

from databroker import Broker
db = Broker.named('temp')

# Insert all metadata/data captured into db.
RE.subscribe(db.insert)

from ophyd.sim import det, motor
from bluesky.plans import scan
dets = [my_sensor_1]   # just one in this case, but it could be more than one

RE(scan(dets, my_three_axis_robot.x, 0, 3, 100))